<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Working with a real world data-set using SQL and Python

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

-   Understand the dataset for Chicago Public School level performance
-   Store the dataset in an Db2 database on IBM Cloud instance
-   Retrieve metadata about tables and columns and query data from mixed case columns
-   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**: 

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_edX/data/Chicago_Public_Schools_-_Progress_Report_Cards__2011-2012-v3.csv">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database


In [6]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
%sql ibm_db_sa://wll66257:09w6lhvc93%40cgmsg@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB

'Connected: wll66257@BLUDB'

In [50]:
import pandas
schools = pandas.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_edX/data/Chicago_Public_Schools_-_Progress_Report_Cards__2011-2012-v3.csv')
p_float = schools['AVERAGE_STUDENT_ATTENDANCE'].str.strip("%").astype(float)/100
schools['AVERAGE_STUDENT_ATTENDANCE']= p_float.round(decimals=2)
schools['AVERAGE_STUDENT_ATTENDANCE']

0      0.96
1      0.96
2      0.96
3      0.96
4      0.93
       ... 
561    0.92
562    0.91
563    0.95
564    0.94
565    0.92
Name: AVERAGE_STUDENT_ATTENDANCE, Length: 566, dtype: float64

In [ ]:
p_float = schools['AVERAGE_STUDENT_ATTENDANCE'].str.strip("%").astype(float)/100
schools['AVERAGE_STUDENT_ATTENDANCE']= p_float.round(decimals=2)
schools['AVERAGE_STUDENT_ATTENDANCE']

In [64]:
%sql PERSIST schools

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB


'Persisted schools'

In [63]:
%sql drop table schools

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


[]

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [61]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='WLL66257'

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
WLL66257,INSTRUCTOR,2021-03-03 15:28:39.702796
WLL66257,INTERNATIONAL_STUDENT_TEST_SCORES,2021-03-03 15:48:14.700008
WLL66257,CHICAGO_SOCIOECONOMIC_DATA,2021-03-03 16:09:23.635408
WLL66257,CHICAGO_PUBLICSCHOOLS_DATA,2021-03-03 19:42:19.477559
WLL66257,SCHOOLS_DATA,2021-03-03 20:04:34.707979
WLL66257,SCHOOLS,2021-03-03 20:05:12.097992


<details><summary>Click here for the solution</summary>

```python
#In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'
```

</details>


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [52]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


1
79


<details><summary>Click here for the solution</summary>

```python
#In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

```

</details>


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [53]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


colname,typename,length
index,BIGINT,8
School ID,BIGINT,8
NAME_OF_SCHOOL,CLOB,1048576
"Elementary, Middle, or High School",CLOB,1048576
Street Address,CLOB,1048576
City,CLOB,1048576
State,CLOB,1048576
ZIP Code,BIGINT,8
Phone Number,CLOB,1048576
Link,CLOB,1048576


<details><summary>Click here for the solution</summary>

```python
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

```

</details>


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [33]:
%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


1
462


<details><summary>Click here for the hint</summary>

```python
Which column specifies the school type e.g. 'ES', 'MS', 'HS'?

Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"

```

</details>


<details><summary>Click here for the solution</summary>

```python
%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

```

</details>


### Problem 2

##### What is the highest Safety Score?


In [34]:
%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


max_safety_score
99.0


<details><summary>Click here for the solution</summary>

```python
# Use the MAX() function

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
    
```

</details>


### Problem 3

##### Which schools have highest Safety Score?


In [35]:
%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,safety_score
Abraham Lincoln Elementary School,99.0
Alexander Graham Bell Elementary School,99.0
Annie Keller Elementary Gifted Magnet School,99.0
Augustus H Burley Elementary School,99.0
Edgar Allan Poe Elementary Classical School,99.0
Edgebrook Elementary School,99.0
Ellen Mitchell Elementary School,99.0
James E McDade Elementary Classical School,99.0
James G Blaine Elementary School,99.0
LaSalle Elementary Language Academy,99.0


<details><summary>Click here for the solution</summary>

```python
# In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
    
```

</details>


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [65]:
%sql select Name_of_School, AVERAGE_STUDENT_ATTENDANCE from SCHOOLS \
    order by AVERAGE_STUDENT_ATTENDANCE desc nulls last limit 10 

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
Annie Keller Elementary Gifted Magnet School,0.98
Edgar Allan Poe Elementary Classical School,0.98
James Ward Elementary School,0.98
John Charles Haines Elementary School,0.98
Orozco Fine Arts & Sciences Elementary School,0.98
Rachel Carson Elementary School,0.98
Albany Park Multicultural Academy,0.97
Ambrose Plamondon Elementary School,0.97
Andrew Jackson Elementary Language Academy,0.97
DeVry University Advantage Academy High School,0.97


<details><summary>Click here for the solution</summary>

```python
%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 
    
```

</details>


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [66]:
%sql SELECT NAME_OF_SCHOOL, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,0.58
Barbara Vick Early Childhood & Family Center,0.61
Dyett High School,0.62
Wendell Phillips Academy High School,0.63
Orr Academy High School,0.66


<details><summary>Click here for the solution</summary>

```python
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

```

</details>


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [67]:
%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,2
Richard T Crane Technical Preparatory High School,5.8E-1
Barbara Vick Early Childhood & Family Center,6.1E-1
Dyett High School,6.2E-1
Wendell Phillips Academy High School,6.3E-1
Orr Academy High School,6.6E-1


<details><summary>Click here for the solution</summary>

```python
#Use the REPLACE() function to replace '%' with ''
#See documentation for this function at: 
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

```

</details>


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [73]:
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL Average_Student_Attendance < 0.7 \
     order by Average_Student_Attendance

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0104N  An unexpected token "Average_Student_Attendance" was found following "where DECIMAL".  Expected tokens may include:  "<space>".  SQLSTATE=42601 SQLCODE=-104
[SQL: SELECT Name_of_School, Average_Student_Attendance        from SCHOOLS       where DECIMAL Average_Student_Attendance < 0.7       order by Average_Student_Attendance]
(Background on this error at: http://sqlalche.me/e/13/f405)


<details><summary>Click here for the hint</summary>

```python
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")

Don't forget the '%' age sign needs to be removed before casting

```

</details>


<details><summary>Click here for the solution</summary>

```python
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

```

</details>


### Problem 8

##### Get the total College Enrollment for each Community Area


In [74]:
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0134N  Improper use of a string column, host variable, constant, or function "COMMUNITY_AREA_NAME".  SQLSTATE=42907 SQLCODE=-134
[SQL: select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT     from SCHOOLS     group by Community_Area_Name]
(Background on this error at: http://sqlalche.me/e/13/f405)


<details><summary>Click here for the hint</summary>

```python
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area

Don't forget to group by the Community Area

```

</details>


<details><summary>Click here for the solution</summary>

```python
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

```

</details>


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [ ]:
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only


<details><summary>Click here for the solution</summary>

```python
# Order the previous query and limit the number of rows you fetch

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

```

</details>


### Problem 10

##### Get the hardship index for the community area which has College Enrollment of 4368


<details><summary>Click here for the solution</summary>

```python
# For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

```

</details>


### Problem 11

##### Get the hardship index for the community area which has the highest value for College Enrollment


<details><summary>Click here for the solution</summary>

```python
# For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )


```

</details>


In [60]:
%sql

 * ibm_db_sa://wll66257:***@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB


'Connected: wll66257@BLUDB'

## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-08-28        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
